# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
#from scipy.misc.pilutil import imread, imresize

import os
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers import Conv3D, MaxPooling3D
#from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

In [2]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


We set the random seed so that the results don't vary drastically.

In [4]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
#tf.set_random_seed(30)
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [5]:
# train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
# val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())
batch_size = 30
train_doc = np.random.permutation(open('Project_data/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/Project_data/val.csv').readlines())
training_folder = 'Project_data/Project_data/train'
testing_folder = 'Project_data/Project_data/test'

image_sequeneces = range(1,30,2)
image_length = 180
image_breadth = 180

input_shape = (len(image_sequeneces), image_length, image_breadth, 3)


#train_doc.shape
# for n in image_sequeneces:
#     print (n)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [6]:
def generator(source_path, folder_list, batch_size, is_train = False, augmention = False, debug=False):
    #print( 'Source path = ', source_path, '; batch size =', batch_size, is_train = False, augmention = False, debug=False)
    img_idx = image_sequeneces
    x = len(img_idx)
    y = image_length
    z = image_breadth
    print ("came here")
    while True:
        t = np.random.permutation(folder_list)
        
        last_batch = 0

        if (len(t)%batch_size) == 0:
            num_batches = int(len(t)/batch_size)
            last_batch = 0
        else:
            last_batch = 
            num_batches = len(t)//batch_size + 1
            

        print ("came here num_batches: " + str(num_batches))

        for batch in range(num_batches): # we iterate over the number of batches
            print ("Inside For")

            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size

                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    if image.shape[1] != image.shape[0]:
                        diff_shape = abs(image.shape[1] - image.shape[0])
                        crop_start = diff_shape // 2
                        crop_end = crop_start + min(image.shape[0],image.shape[1])
                        if image.shape[1] > image.shape[0]:
                            image = image[:, crop_start:crop_end]
                        else:
                            image = image[crop_start:crop_end, :]
                            
                    resized_image = resize(image, output_shape=(y,z))
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = resized_image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = resized_image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = resized_image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
# train_path = '/notebooks/storage/Final_data/Collated_training/train'
# val_path = '/notebooks/storage/Final_data/Collated_training/val'
train_path = 'Project_data/Project_data/train'
val_path = 'Project_data/Project_data/val'
#Project_data/Project_data
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [9]:


#write your model here
model = Sequential()
model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling3D(pool_size=2))

model.add(Conv3D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))



C:\Users\mathu\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [10]:
optimiser = optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                      │ (None, 13, 178, 178, 32)    │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d (MaxPooling3D)         │ (None, 6, 89, 89, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_1 (Conv3D)                    │ (None, 4, 87, 87, 64)       │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_1 (MaxPooling3D)       │ (None, 2, 43, 43, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 236672)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      60,588,288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │           1,285 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 60,647,557 (231.35 MB)

 Trainable params: 60,647,557 (231.35 MB)

 Non-trainable params: 0 (0.00 B)

None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [11]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [12]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5' + '.keras'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')
#checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [14]:

# history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None,  initial_epoch=0)

came here
came here num_batches: 23
Inside For


C:\Users\mathu\AppData\Local\Temp\ipykernel_29908\3414696527.py:28: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Inside For
Epoch 1/10
 1/23 ━━━━━━━━━━━━━━━━━━━━ 2:14 6s/step - categorical_accuracy: 0.1667 - loss: 1.6192Inside For
 2/23 ━━━━━━━━━━━━━━━━━━━━ 1:17 4s/step - categorical_accuracy: 0.1750 - loss: 1427.8026Inside For
 3/23 ━━━━━━━━━━━━━━━━━━━━ 1:36 5s/step - categorical_accuracy: 0.1759 - loss: 1639.2078Inside For
 4/23 ━━━━━━━━━━━━━━━━━━━━ 1:40 5s/step - categorical_accuracy: 0.1861 - loss: 1616.1344Inside For
 5/23 ━━━━━━━━━━━━━━━━━━━━ 1:40 6s/step - categorical_accuracy: 0.1876 - loss: 1545.1902Inside For
 6/23 ━━━━━━━━━━━━━━━━━━━━ 1:35 6s/step - categorical_accuracy: 0.1878 - loss: 1463.1887Inside For
 7/23 ━━━━━━━━━━━━━━━━━━━━ 1:31 6s/step - categorical_accuracy: 0.1888 - loss: 1383.1556Inside For
 8/23 ━━━━━━━━━━━━━━━━━━━━ 1:25 6s/step - categorical_accuracy: 0.1908 - loss: 1309.0474Inside For
 9/23 ━━━━━━━━━━━━━━━━━━━━ 1:20 6s/step - categorical_accuracy: 0.1922 - loss: 1241.6708Inside For
10/23 ━━━━━━━━━━━━━━━━━━━━ 1:14 6s/step - categorical_accuracy: 0.1933 - loss: 1180.7589In

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
IndexError: index 663 is out of bounds for axis 0 with size 663
Traceback (most recent call last):

  File "C:\Users\mathu\anaconda3\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 269, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "C:\Users\mathu\anaconda3\Lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\mathu\anaconda3\Lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\mathu\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\generator_data_adapter.py", line 52, in get_tf_iterator
    for batch in self.generator:

  File "C

In [133]:
# testing generative 
test_gen = generator(train_path, train_doc, 1, is_train = True, augmention = True, debug = True)
d = next(test_gen)[0]
print (d)

came here
came here num_batches: 663
Inside For
Inside For1
[[[[[0.64870387 0.6447823  0.62909603]
    [0.64795512 0.64403355 0.62834728]
    [0.64038557 0.636464   0.62077773]
    ...
    [0.54945314 0.53831714 0.50363094]
    [0.54582292 0.53137422 0.49680436]
    [0.55663896 0.5298081  0.49865705]]

   [[0.64221895 0.63829738 0.62261111]
    [0.64074218 0.63682061 0.62113434]
    [0.64471126 0.64078969 0.62510341]
    ...
    [0.54574209 0.53486347 0.50674641]
    [0.54836541 0.53187376 0.50646603]
    [0.55567157 0.52842122 0.50508982]]

   [[0.60596079 0.60203922 0.58635294]
    [0.60877436 0.6048528  0.58916652]
    [0.61176664 0.60784507 0.59215879]
    ...
    [0.54519248 0.53559572 0.50792533]
    [0.55185473 0.52755648 0.51211512]
    [0.55778521 0.52727568 0.51425195]]

   ...

   [[0.47181451 0.4718982  0.46586078]
    [0.47374642 0.47514617 0.46210119]
    [0.43012285 0.4305912  0.42127553]
    ...
    [0.11782449 0.12174606 0.09841519]
    [0.12868647 0.12560396 0.1057863

C:\Users\mathu\AppData\Local\Temp\ipykernel_32208\2174006485.py:29: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


In [125]:
for i in range(1):
    print (i)

0
